In [ ]:
# make sure to install these packages before running:
!pip install pandas
!pip install sodapy
!pip install python-dotenv

In [8]:
import pandas as pd
from sodapy import Socrata
from dotenv import load_dotenv
import os
import json
import hashlib

## Data Collection and Acquisition

In [10]:
#load secrets

load_dotenv(override = True)

api_token = os.getenv("API_TOKEN")
email = os.getenv("EMAIL")
password = os.getenv("PASSWORD")

In [11]:
#API authentication for the datasets
client = Socrata("data.ny.gov",
                 api_token,
                 username= email,
                 password= password)

**Dataset 1**

In [12]:
# returns json response - DATASET 1 “MTA Subway Trains Delayed: Beginning 2020”
results = client.get_all("g937-7k7c", 
    where="month >= '2024-01' AND month <= '2024-12'")

In [13]:
# Convert to pandas DataFrame - DATASET 1 “MTA Subway Trains Delayed: Beginning 2020”
delayed_df = pd.DataFrame.from_records(results)
delayed_df

,month,division,line,day_type,incidents,reporting_category
0,2024-01-01T00:00:00.000,A DIVISION,1,1,0,NaN
1,2024-01-01T00:00:00.000,A DIVISION,1,1,31,Crew Availability
2,2024-01-01T00:00:00.000,A DIVISION,1,1,2,External Factors
3,2024-01-01T00:00:00.000,A DIVISION,1,1,92,Infrastructure & Equipment
4,2024-01-01T00:00:00.000,A DIVISION,1,1,86,Operating Conditions
...,...,...,...,...,...,...
3761,2024-12-01T00:00:00.000,B DIVISION,S Rock,2,16,External Factors
3762,2024-12-01T00:00:00.000,B DIVISION,S Rock,2,6,Infrastructure & Equipment
3763,2024-12-01T00:00:00.000,B DIVISION,S Rock,2,1,Operating Conditions
3764,2024-12-01T00:00:00.000,B DIVISION,S Rock,2,1,Planned ROW Work


**Dataset 1 - SHA256 Integrity Check**

In [14]:
# Write the data to disk as a CSV
delayed_df.to_csv("raw_data/delayed.csv", index=False)

# Read the CSV as bytes
with open("raw_data/delayed.csv", "rb") as f:
    data_bytes = f.read()

# Calculate SHA-256 of uncompressed data
sha256_hash = hashlib.sha256(data_bytes).hexdigest()

# Write the hex digest
with open("SHA256/delayed.sha", "w") as f:
    f.write(sha256_hash)

**Dataset 1 - Cleaning and Quality Assessment BEFORE INTEGRATION**

In [16]:
#Drop NaN Values
delayed_df = delayed_df.dropna()
delayed_df

,month,division,line,day_type,incidents,reporting_category
1,2024-01-01T00:00:00.000,A DIVISION,1,1,31,Crew Availability
2,2024-01-01T00:00:00.000,A DIVISION,1,1,2,External Factors
3,2024-01-01T00:00:00.000,A DIVISION,1,1,92,Infrastructure & Equipment
4,2024-01-01T00:00:00.000,A DIVISION,1,1,86,Operating Conditions
5,2024-01-01T00:00:00.000,A DIVISION,1,1,49,Planned ROW Work
...,...,...,...,...,...,...
3761,2024-12-01T00:00:00.000,B DIVISION,S Rock,2,16,External Factors
3762,2024-12-01T00:00:00.000,B DIVISION,S Rock,2,6,Infrastructure & Equipment
3763,2024-12-01T00:00:00.000,B DIVISION,S Rock,2,1,Operating Conditions
3764,2024-12-01T00:00:00.000,B DIVISION,S Rock,2,1,Planned ROW Work


In [17]:
#Check Data Types -- originally all were objects! 
delayed_df.dtypes

month                 object
division              object
line                  object
day_type              object
incidents             object
reporting_category    object
dtype: object

In [18]:
#Convert to proper data types
delayed_df['month'] = pd.to_datetime(delayed_df['month']).dt.normalize()
delayed_df['division'] = delayed_df['division'].astype(str)
delayed_df['line'] = delayed_df['line'].astype(str)
delayed_df['day_type'] = delayed_df['day_type'].astype(str)
delayed_df['incidents'] = delayed_df['incidents'].astype(int)
delayed_df['reporting_category'] = delayed_df['reporting_category'].astype(str)

In [19]:
#Clean string columns for synactic issues (capitalization and white space from front and back)
delayed_df['division'] = delayed_df['division'].str.strip().str.upper()
delayed_df['line'] = delayed_df['line'].str.strip().str.upper()
delayed_df['day_type'] = delayed_df['day_type'].str.strip().str.upper()
delayed_df['reporting_category'] = delayed_df['reporting_category'].str.strip().str.upper()

In [20]:
#Semantic check. Checking no numbers below 0 for incidents column. 
delayed_df[delayed_df['incidents'] < 0]

,month,division,line,day_type,incidents,reporting_category


In [21]:
#Syntactic check, make sure all the months are formatted the same.
delayed_df['month'].dt.strftime('%Y-%m-%d').unique()

array(['2024-01-01', '2024-02-01', '2024-03-01', '2024-04-01',
       '2024-05-01', '2024-06-01', '2024-07-01', '2024-08-01',
       '2024-09-01', '2024-10-01', '2024-11-01', '2024-12-01'],
      dtype=object)

In [22]:
#Semantic check. Make sure all Division A are associated with numbered subway lines and 
#all Division B are associated with lettered subway lines

bad_A = delayed_df[
    (delayed_df['division'] == 'A DIVISION') &
    ~delayed_df['line'].str.contains(r'\d', na=False)      
]
bad_B = delayed_df[
    (delayed_df['division'] == 'B DIVISION') &
    ~delayed_df['line'].str.fullmatch(r'[A-Z ]+')  
]

print("A DIVISION with non-numbered lines:")
print(bad_A)

print("\nB DIVISION with non-lettered lines:")
print(bad_B)

A DIVISION with non-numbered lines:
Empty DataFrame
Columns: [month, division, line, day_type, incidents, reporting_category]
Index: []

B DIVISION with non-lettered lines:
Empty DataFrame
Columns: [month, division, line, day_type, incidents, reporting_category]
Index: []


In [23]:
#Check for duplicate rows across the entire dataset
delayed_df[delayed_df.duplicated()]

,month,division,line,day_type,incidents,reporting_category


In [24]:
# checking for syntactic and semantic errors syntax issues
print("Lines:", delayed_df['line'].unique())
print("Divisions:", delayed_df['division'].unique())
print("Day Types:", delayed_df['day_type'].unique())
print("Reporting Categories:", delayed_df['reporting_category'].unique())

# so only going to do analysis on lines: 1', '2', '3', '4', '5', '6', '7', 'S 42ND', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'JZ', 'L', 'M', 'N', 'Q', 'R', 'S FKLN', 'S ROCK' ONLY
#because all these lines can be found in both datasets. 

Lines: ['1' '2' '3' '4' '5' '6' '7' 'S 42ND' 'A' 'B' 'C' 'D' 'E' 'F' 'G' 'JZ' 'L'
 'M' 'N' 'Q' 'R' 'S FKLN' 'S ROCK']
Divisions: ['A DIVISION' 'B DIVISION']
Day Types: ['1' '2']
Reporting Categories: ['CREW AVAILABILITY' 'EXTERNAL FACTORS' 'INFRASTRUCTURE & EQUIPMENT'
 'OPERATING CONDITIONS' 'PLANNED ROW WORK' 'POLICE & MEDICAL']


In [25]:
delayed_df

,month,division,line,day_type,incidents,reporting_category
1,2024-01-01,A DIVISION,1,1,31,CREW AVAILABILITY
2,2024-01-01,A DIVISION,1,1,2,EXTERNAL FACTORS
3,2024-01-01,A DIVISION,1,1,92,INFRASTRUCTURE & EQUIPMENT
4,2024-01-01,A DIVISION,1,1,86,OPERATING CONDITIONS
5,2024-01-01,A DIVISION,1,1,49,PLANNED ROW WORK
...,...,...,...,...,...,...
3761,2024-12-01,B DIVISION,S ROCK,2,16,EXTERNAL FACTORS
3762,2024-12-01,B DIVISION,S ROCK,2,6,INFRASTRUCTURE & EQUIPMENT
3763,2024-12-01,B DIVISION,S ROCK,2,1,OPERATING CONDITIONS
3764,2024-12-01,B DIVISION,S ROCK,2,1,PLANNED ROW WORK


**Dataset 2**

In [26]:
# returns json response - DATASET 2 "MTA Subway Customer Journey-Focused Metrics: 2020-2024”
results = client.get_all("4apg-4kt9", 
    where="month >= '2024-01' AND month <= '2024-12'")

In [27]:
# Convert to pandas DataFrame - DATASET 2 "MTA Subway Customer Journey-Focused Metrics: 2020-2024”
metrics_df = pd.DataFrame.from_records(results)
metrics_df

,month,division,line,period,num_passengers,additional_platform_time,additional_train_time,total_apt,total_att,over_five_mins,over_five_mins_perc,customer_journey_time
0,2024-01-01T00:00:00.000,A DIVISION,1,offpeak,4873143.5,1.656052,0.79665726,8070179.0,3882225.2,763100.9,0.15659314,0.84340686
1,2024-01-01T00:00:00.000,A DIVISION,1,peak,4765373.0,2.115036,0.8733,10078935.0,4161600.2,743524.6,0.15602654,0.84397346
2,2024-01-01T00:00:00.000,A DIVISION,2,offpeak,2878163.8,1.6068327,0.50131786,4624727.5,1442874.9,487204.1,0.16927601,0.830724
3,2024-01-01T00:00:00.000,A DIVISION,2,peak,3092725.5,2.0733223,0.5617463,6412216.5,1737327.1,460791.2,0.14899196,0.85100806
4,2024-01-01T00:00:00.000,A DIVISION,3,offpeak,2125353.8,1.2491903,0.3281006,2654971.2,697329.8,265963.7,0.12513857,0.8748614
...,...,...,...,...,...,...,...,...,...,...,...,...
571,2024-12-01T00:00:00.000,B DIVISION,S Fkln,peak,194189.34,0.5707303,0.057167325,110829.74,11101.285,9461.589,0.048723523,0.9512765
572,2024-12-01T00:00:00.000,B DIVISION,S Rock,offpeak,26513.352,1.2623241,-0.90806454,33468.44,-24075.834,3445.474,0.12995242,0.87004757
573,2024-12-01T00:00:00.000,B DIVISION,S Rock,peak,19257.482,0.8787027,-1.183044,16921.602,-22782.447,1812.6135,0.09412515,0.90587485
574,2024-12-01T00:00:00.000,B DIVISION,W,offpeak,1265461.6,0.9385583,0.2576316,1187709.5,326022.94,142330.02,0.112472795,0.8875272


**Dataset 2 - SHA256 Integrity Check**

In [28]:
# Write the data to disk as a CSV
metrics_df.to_csv("raw_data/metric.csv", index=False)

# Read the CSV as bytes
with open("raw_data/metric.csv", "rb") as f:
    data_bytes = f.read()

# Calculate SHA-256 of uncompressed data
sha256_hash = hashlib.sha256(data_bytes).hexdigest()

# Write the hex digest
with open("SHA256/metric.sha", "w") as f:
    f.write(sha256_hash)

**Dataset 2 - Cleaning and Quality Assessment BEFORE INTEGRATION**

In [41]:
#Drop NaN Values
metrics_df = metrics_df.dropna()
metrics_df

,month,division,line,period,num_passengers,additional_platform_time,additional_train_time,total_apt,total_att,over_five_mins,over_five_mins_perc,customer_journey_time,num_passenger
0,2024-01-01,A DIVISION,1,OFFPEAK,4873143.5,1.656052,0.796657,8.070179e+06,3882225.200,763100.9000,0.156593,0.843407,4873143.500
1,2024-01-01,A DIVISION,1,PEAK,4765373.0,2.115036,0.873300,1.007894e+07,4161600.200,743524.6000,0.156027,0.843973,4765373.000
2,2024-01-01,A DIVISION,2,OFFPEAK,2878163.8,1.606833,0.501318,4.624728e+06,1442874.900,487204.1000,0.169276,0.830724,2878163.800
3,2024-01-01,A DIVISION,2,PEAK,3092725.5,2.073322,0.561746,6.412216e+06,1737327.100,460791.2000,0.148992,0.851008,3092725.500
4,2024-01-01,A DIVISION,3,OFFPEAK,2125353.8,1.249190,0.328101,2.654971e+06,697329.800,265963.7000,0.125139,0.874861,2125353.800
...,...,...,...,...,...,...,...,...,...,...,...,...,...
571,2024-12-01,B DIVISION,S FKLN,PEAK,194189.34,0.570730,0.057167,1.108297e+05,11101.285,9461.5890,0.048724,0.951276,194189.340
572,2024-12-01,B DIVISION,S ROCK,OFFPEAK,26513.352,1.262324,-0.908065,3.346844e+04,-24075.834,3445.4740,0.129952,0.870048,26513.352
573,2024-12-01,B DIVISION,S ROCK,PEAK,19257.482,0.878703,-1.183044,1.692160e+04,-22782.447,1812.6135,0.094125,0.905875,19257.482
574,2024-12-01,B DIVISION,W,OFFPEAK,1265461.6,0.938558,0.257632,1.187710e+06,326022.940,142330.0200,0.112473,0.887527,1265461.600


In [42]:
# Check data types
metrics_df.dtypes

month                       datetime64[ns]
division                            object
line                                object
period                              object
num_passengers                      object
additional_platform_time           float64
additional_train_time              float64
total_apt                          float64
total_att                          float64
over_five_mins                     float64
over_five_mins_perc                float64
customer_journey_time              float64
num_passenger                      float64
dtype: object

In [43]:
# Convert to correct data types
metrics_df['month'] = pd.to_datetime(metrics_df['month']).dt.normalize()
metrics_df['division'] = metrics_df['division'].astype(str)
metrics_df['line'] = metrics_df['line'].astype(str)
metrics_df['period'] = metrics_df['period'].astype(str)
metrics_df['num_passenger'] = metrics_df['num_passengers'].astype(float)
metrics_df['additional_platform_time'] = metrics_df['additional_platform_time'].astype(float)
metrics_df['additional_train_time'] = metrics_df['additional_train_time'].astype(float)
metrics_df['total_apt'] = metrics_df['total_apt'].astype(float)
metrics_df['total_att'] = metrics_df['total_att'].astype(float)
metrics_df['over_five_mins'] = metrics_df['over_five_mins'].astype(float)
metrics_df['over_five_mins_perc'] = metrics_df['over_five_mins_perc'].astype(float)
metrics_df['customer_journey_time'] = metrics_df['customer_journey_time'].astype(float)

In [44]:
#Clean string columns for synactic issues (capitalization and white space from front and back)
metrics_df['line'] = metrics_df['line'].str.strip().str.upper()
metrics_df['division'] = metrics_df['division'].str.strip().str.upper()
metrics_df['period'] = metrics_df['period'].str.strip().str.upper()

In [45]:
#Semantic check. Checking for negative times for numeric columns. 
cols_to_check = [
    'num_passenger', 'additional_platform_time', 'additional_train_time',
    'total_apt', 'total_att', 'over_five_mins',
    'over_five_mins_perc', 'customer_journey_time'
]

for col in cols_to_check:
    if (metrics_df[col] < 0).any():
        print(col)

additional_train_time
total_att


In [46]:
#Syntactic check, make sure all the months are formatted the same.
metrics_df['month'].dt.strftime('%Y-%m-%d').unique()

array(['2024-01-01', '2024-02-01', '2024-03-01', '2024-04-01',
       '2024-05-01', '2024-06-01', '2024-07-01', '2024-08-01',
       '2024-09-01', '2024-10-01', '2024-11-01', '2024-12-01'],
      dtype=object)

In [47]:
#Semantic check. Make sure all Division A are associated with numbered subway lines and 
#all Division B are associated with lettered subway lines

bad_A_2 = metrics_df[
    (metrics_df['division'] == 'A DIVISION') &
    ~metrics_df['line'].str.contains(r'\d', na=False)  
]
bad_B_2 = metrics_df[
    (metrics_df['division'] == 'B DIVISION') &
    ~metrics_df['line'].str.fullmatch(r'[A-Z ]+')  
]

print("A DIVISION with non-numbered lines:")
print(bad_A_2)

print("\nB DIVISION with non-lettered lines:")
print(bad_B_2)

A DIVISION with non-numbered lines:
Empty DataFrame
Columns: [month, division, line, period, num_passengers, additional_platform_time, additional_train_time, total_apt, total_att, over_five_mins, over_five_mins_perc, customer_journey_time, num_passenger]
Index: []

B DIVISION with non-lettered lines:
Empty DataFrame
Columns: [month, division, line, period, num_passengers, additional_platform_time, additional_train_time, total_apt, total_att, over_five_mins, over_five_mins_perc, customer_journey_time, num_passenger]
Index: []


In [48]:
#Check for duplicate rows across the entire dataset
metrics_df[metrics_df.duplicated()]

,month,division,line,period,num_passengers,additional_platform_time,additional_train_time,total_apt,total_att,over_five_mins,over_five_mins_perc,customer_journey_time,num_passenger


In [49]:
# checking for syntactic and semantic errors syntax issues
print("Lines:", metrics_df['line'].unique())
print("Divisions:", metrics_df['division'].unique())
print("Period:", metrics_df['period'].unique())

Lines: ['1' '2' '3' '4' '5' '6' '7' 'S 42ND' 'A' 'B' 'C' 'D' 'E' 'F' 'G' 'JZ' 'L'
 'M' 'N' 'Q' 'R' 'S FKLN' 'S ROCK' 'W']
Divisions: ['A DIVISION' 'B DIVISION']
Period: ['OFFPEAK' 'PEAK']


In [50]:
metrics_df

,month,division,line,period,num_passengers,additional_platform_time,additional_train_time,total_apt,total_att,over_five_mins,over_five_mins_perc,customer_journey_time,num_passenger
0,2024-01-01,A DIVISION,1,OFFPEAK,4873143.5,1.656052,0.796657,8.070179e+06,3882225.200,763100.9000,0.156593,0.843407,4873143.500
1,2024-01-01,A DIVISION,1,PEAK,4765373.0,2.115036,0.873300,1.007894e+07,4161600.200,743524.6000,0.156027,0.843973,4765373.000
2,2024-01-01,A DIVISION,2,OFFPEAK,2878163.8,1.606833,0.501318,4.624728e+06,1442874.900,487204.1000,0.169276,0.830724,2878163.800
3,2024-01-01,A DIVISION,2,PEAK,3092725.5,2.073322,0.561746,6.412216e+06,1737327.100,460791.2000,0.148992,0.851008,3092725.500
4,2024-01-01,A DIVISION,3,OFFPEAK,2125353.8,1.249190,0.328101,2.654971e+06,697329.800,265963.7000,0.125139,0.874861,2125353.800
...,...,...,...,...,...,...,...,...,...,...,...,...,...
571,2024-12-01,B DIVISION,S FKLN,PEAK,194189.34,0.570730,0.057167,1.108297e+05,11101.285,9461.5890,0.048724,0.951276,194189.340
572,2024-12-01,B DIVISION,S ROCK,OFFPEAK,26513.352,1.262324,-0.908065,3.346844e+04,-24075.834,3445.4740,0.129952,0.870048,26513.352
573,2024-12-01,B DIVISION,S ROCK,PEAK,19257.482,0.878703,-1.183044,1.692160e+04,-22782.447,1812.6135,0.094125,0.905875,19257.482
574,2024-12-01,B DIVISION,W,OFFPEAK,1265461.6,0.938558,0.257632,1.187710e+06,326022.940,142330.0200,0.112473,0.887527,1265461.600


**Dataset 3**

In [39]:
# returns json response - DATASET 3 “MTA Subway Service Delivered: 2020-2024”
results = client.get_all("bg59-42xi",
    where="month >= '2024-01' AND month <= '2024-12'")

In [17]:
# Convert to pandas DataFrame - DATASET 3 “MTA Subway Service Delivered: 2020-2024”
delivered_df = pd.DataFrame.from_records(results)
delivered_df

,month,division,line,day_type,num_sched_trains,num_actual_trains,service_delivered
0,2024-01-01T00:00:00.000,A DIVISION,1,1,1696,1499,0.8838443
1,2024-01-01T00:00:00.000,A DIVISION,1,2,864,819,0.9479167
2,2024-01-01T00:00:00.000,A DIVISION,2,1,2244,2027,0.90329766
3,2024-01-01T00:00:00.000,A DIVISION,2,2,1117,1067,0.95523727
4,2024-01-01T00:00:00.000,A DIVISION,3,1,2069,1937,0.93620104
...,...,...,...,...,...,...,...
549,2024-12-01T00:00:00.000,B DIVISION,Q,2,1756,1714,0.976082
550,2024-12-01T00:00:00.000,B DIVISION,R,1,2163,1611,0.7447989
551,2024-12-01T00:00:00.000,B DIVISION,R,2,1360,1308,0.9617647
552,2024-12-01T00:00:00.000,B DIVISION,W,1,819,749,0.9145299


**Dataset 3 - SHA256 Integrity Check**

In [18]:
# Write the data to disk as a CSV
delivered_df.to_csv("raw_data/delivered.csv", index=False)

# Read the CSV as bytes
with open("raw_data/delivered.csv", "rb") as f:
    data_bytes = f.read()

# Calculate SHA-256 of uncompressed data
sha256_hash = hashlib.sha256(data_bytes).hexdigest()

# Write the hex digest
with open("SHA256/delivered.sha", "w") as f:
    f.write(sha256_hash)

**Dataset 3 - Cleaning BEFORE INTEGRATION**

In [19]:
#Clean line column - DATASET 3 “MTA Subway Service Delivered: 2020-2024”
delivered_df['line'] = delivered_df['line'].str.strip().str.upper()

In [20]:
delivered_df['line'].unique()

#drop the lines where they dont have in common (W,H) or not known (GS, and FS no clue what they are even after googling)

array(['1', '2', '3', '4', '5', '6', '7', 'S 42ND', 'A', 'B', 'C', 'D',
       'E', 'F', 'G', 'JZ', 'L', 'M', 'N', 'Q', 'R', 'S FKLN', 'S ROCK',
       'W', 'GS', 'FS', 'H'], dtype=object)

In [21]:
#drop line W, H, GS, and FS to remain consistent between all three datasets
delivered_df = delivered_df[
    (delivered_df['line'] != 'W') &
    (delivered_df['line'] != 'H') &
    (delivered_df['line'] != 'GS') &
    (delivered_df['line'] != 'FS')
]
delivered_df

,month,division,line,day_type,num_sched_trains,num_actual_trains,service_delivered
0,2024-01-01T00:00:00.000,A DIVISION,1,1,1696,1499,0.8838443
1,2024-01-01T00:00:00.000,A DIVISION,1,2,864,819,0.9479167
2,2024-01-01T00:00:00.000,A DIVISION,2,1,2244,2027,0.90329766
3,2024-01-01T00:00:00.000,A DIVISION,2,2,1117,1067,0.95523727
4,2024-01-01T00:00:00.000,A DIVISION,3,1,2069,1937,0.93620104
...,...,...,...,...,...,...,...
547,2024-12-01T00:00:00.000,B DIVISION,N,2,1599,1544,0.96560353
548,2024-12-01T00:00:00.000,B DIVISION,Q,1,2163,1989,0.9195562
549,2024-12-01T00:00:00.000,B DIVISION,Q,2,1756,1714,0.976082
550,2024-12-01T00:00:00.000,B DIVISION,R,1,2163,1611,0.7447989


**Dataset 4**

In [22]:
# returns json response - DATASET 4 "MTA Service Alerts: Beginning April 2020"
results = client.get_all("7kct-peq7",
    where= "date >= '2024-01-01T00:00:00' AND date < '2025-01-01T00:00:00'"
)

In [23]:
# Convert to pandas DataFrame - DATASET 4 "MTA Service Alerts: Beginning April 2020"
alerts_df = pd.DataFrame.from_records(results)
alerts_df

,alert_id,event_id,update_number,date,agency,status_label,affected,header,description
0,281016,134282,2,2024-01-01T00:01:00.000,NYCT Bus,delays,SIM1C,The following SIM1C trips will not run:\n\nFro...,We’re running as much service as we can with t...
1,281017,134319,0,2024-01-01T00:11:00.000,NYCT Subway,delays,J,Jamaica Center-Parsons/Archer-bound J trains a...,NaN
2,281018,134320,0,2024-01-01T00:12:00.000,NYCT Subway,delays,A,Northbound A trains are running with delays af...,NaN
3,281019,134321,0,2024-01-01T00:20:00.000,NYCT Subway,local-to-express,C,Northbound C trains are running express from 1...,"For service to/from 135 St, take a southbound ..."
4,281020,134316,5,2024-01-01T00:24:00.000,LIRR,some-delays,City Terminal Zone,There are scattered delays along the City Term...,NaN
...,...,...,...,...,...,...,...,...,...
92281,379670,184992,2,2024-12-31T23:33:00.000,BT,some-delays,Bronx-Whitestone Bridge,BWB: Disabled vehicle; Queens bound cleared.,NaN
92282,379671,184993,0,2024-12-31T23:48:00.000,NYCT Subway,delays,4,Northbound 4 trains are delayed while we reque...,NaN
92283,379672,184994,0,2024-12-31T23:53:00.000,NYCT Subway,delays,2 | 5,Northbound 2/5 trains are delayed while we req...,NaN
92284,379673,184993,1,2024-12-31T23:55:00.000,NYCT Subway,delays,4,Northbound 4 trains are running with delays af...,NaN


**Dataset 4 - SHA256 Integrity Check**

In [24]:
# Write the data to disk as a CSV
alerts_df.to_csv("raw_data/alerts.csv", index=False)

# Read the CSV as bytes
with open("raw_data/alerts.csv", "rb") as f:
    data_bytes = f.read()

# Calculate SHA-256 of uncompressed data
sha256_hash = hashlib.sha256(data_bytes).hexdigest()

# Write the hex digest
with open("SHA256/alerts.sha", "w") as f:
    f.write(sha256_hash)

**Dataset 4 - Cleaning**

In [25]:
alerts_df['affected'].unique()

array(['SIM1C', 'J', 'A', ..., 'M55 | M8',
       'B36 | B41 | B69 | X27 | X28 | X37 | X38',
       'BX10 | BX13 | BX15 | BX2 | BX3'], dtype=object)